In [1]:
import os
print(os.getcwd())  # this is where Jupyter is looking from
#one folder back
os.chdir('..')
print(os.getcwd())

/Users/christianhellum/cand_merc/n1_masters_thesis/model_training
/Users/christianhellum/cand_merc/n1_masters_thesis


In [13]:
import pandas as pd
import numpy as np
from load_data import load_dataset, ImageDataset

from split import get_dataloaders
from custom_data_aug import get_transforms

import torch
import torch.nn as nn
import torchvision
from config import CFG

from torchinfo import summary

from torch.utils.tensorboard import SummaryWriter
from training import train  # or wherever your updated function lives

from acc_loss_plot import plot_training_curves

from torchvision.models import VGG16_Weights

In [3]:
# Load the dataframe
csv_path = "C:/Users/Christian/Desktop/N1_data/filtered_df.csv"
image_dir = "C:/Users/Christian/Desktop/N1_data/image_data/"
df, idx_to_label = load_dataset(csv_path, image_dir)

In [4]:
print(df.head())

                                           file_name label  label_idx
0  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
1  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
2  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
3  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
4  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0


In [5]:
train_transforms, val_transforms = get_transforms()

train_dataset, val_dataset, train_loader, val_loader = get_dataloaders(
    df, train_transforms, val_transforms
)

In [ ]:
def build_model(device: torch.device, num_classes: int = CFG.NUM_CLASSES) -> nn.Module:
    # Set seeds
    torch.manual_seed(CFG.SEED)
    torch.cuda.manual_seed(CFG.SEED)
    
    model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    for layer in list(model.features.children())[-4:]:
        for param in layer.parameters():
            param.requires_grad = True

    model.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    model.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512, 128),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(128, num_classes)
    ).to(device)

    return model

In [25]:
cnn = build_model(device=CFG.DEVICE)

# View model summary
summary(
    model=cnn, 
    input_size=(CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
VGG (VGG)                                [64, 3, 224, 224]    [64, 34]             --                   Partial
├─Sequential (features)                  [64, 3, 224, 224]    [64, 512, 7, 7]      --                   Partial
│    └─Conv2d (0)                        [64, 3, 224, 224]    [64, 64, 224, 224]   (1,792)              False
│    └─ReLU (1)                          [64, 64, 224, 224]   [64, 64, 224, 224]   --                   --
│    └─Conv2d (2)                        [64, 64, 224, 224]   [64, 64, 224, 224]   (36,928)             False
│    └─ReLU (3)                          [64, 64, 224, 224]   [64, 64, 224, 224]   --                   --
│    └─MaxPool2d (4)                     [64, 64, 224, 224]   [64, 64, 112, 112]   --                   --
│    └─Conv2d (5)                        [64, 64, 112, 112]   [64, 128, 112, 112]  (73,856)             False
│    └─ReLU

In [12]:
from torchvision.models import VGG16_Weights
print(VGG16_Weights.DEFAULT)
print(list(VGG16_Weights))


VGG16_Weights.IMAGENET1K_V1
[VGG16_Weights.IMAGENET1K_V1, VGG16_Weights.IMAGENET1K_FEATURES]


In [8]:
# Define Loss Function
loss_fn = nn.CrossEntropyLoss(
    label_smoothing=0.1
)

# Define Optimizer
optimizer = torch.optim.Adam(
    cnn.parameters(),
    lr=CFG.LR
)

In [9]:
writer = SummaryWriter(log_dir="runs/exp1_efficientnet")

session = train(
    model=cnn,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=CFG.EPOCHS,
    device=CFG.DEVICE,
    writer=writer
)

writer.close()


Epoch 1/10


  0%|          | 0/117 [00:00<?, ?it/s]

: 

In [ ]:
# Convert history dict to DataFrame
session_history_df = pd.DataFrame(session_history)
session_history_df

In [ ]:
from acc_loss_plot import plot_training_curves
# Plot EfficientNet session training history 
plot_training_curves(session_history)